In [110]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm

## lấy mfcc của file wav

In [111]:
# read file
def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

## lấy mfcc của tất cả các file trong dir

In [112]:
# lấy mfcc của tất cả các file wav trong wav
def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    return mfcc

## Hàm Clustering

In [113]:
def clustering(X, n_clusters=14):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans

## Data

In [114]:
class_names = ['cothe', 'da', 'duoc', 'nguoi','trong']
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join(cname))

Load cothe dataset
Load da dataset
Load duoc dataset
Load nguoi dataset
Load trong dataset


In [115]:
print(len(dataset['cothe']))
print(len(dataset['da']))
print(len(dataset['duoc']))
print(len(dataset['nguoi']))
print(len(dataset['trong']))

124
99
100
100
100


## split train test

In [116]:
trainset = {}
testset = {}
n_test = {'cothe': 0, 'da': 0, 'duoc': 0, 'nguoi': 0,'trong': 0}
for cname in class_names:
    n = len(dataset[cname])
    n_train = math.floor(n*0.8)
    trainset[cname] = dataset[cname][:n_train]
    testset[cname] = dataset[cname][n_train:]
    n_test[cname] += len(testset[cname])
    
print(len(trainset['cothe']))
print(len(trainset['da']))
print(len(trainset['duoc']))
print(len(trainset['nguoi']))
print(len(trainset['trong']))

99
79
80
80
80


## Fit kmeans trên tập train

In [117]:
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in trainset.items()], axis=0)
print("vectors", all_vectors.shape)
# Run K-Means algorithm to get clusters
kmeans = clustering(all_vectors)
print("centers", kmeans.cluster_centers_.shape)
print(kmeans)

vectors (11574, 36)
centers (14, 36)
centers (14, 36)
KMeans(n_clusters=14, n_init=50, random_state=0)


In [118]:
for cname in class_names:
    trainset[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in trainset[cname]])
    testset[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in testset[cname]])

    

# Train

In [119]:
models = {}

## Models cho 'có thể' 5x3

In [120]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=4*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])


Xbn = np.concatenate(trainset['cothe'])
lengths = list([len(x) for x in trainset['cothe']])
print("training class", 'cothe')
print(Xbn.shape, lengths, len(lengths))
hmm.fit(Xbn, lengths=lengths)
models['cothe'] = hmm
print("Training done")

training class cothe
(4501, 1) [34, 31, 56, 39, 44, 40, 46, 35, 45, 47, 37, 56, 36, 31, 39, 35, 42, 30, 30, 37, 36, 39, 26, 34, 47, 36, 41, 34, 26, 30, 25, 27, 16, 39, 27, 52, 25, 27, 20, 22, 36, 53, 44, 44, 28, 57, 53, 36, 37, 30, 34, 38, 51, 43, 40, 47, 38, 44, 102, 44, 52, 56, 35, 37, 39, 25, 41, 52, 43, 33, 49, 59, 35, 42, 40, 37, 40, 35, 36, 28, 45, 44, 23, 30, 48, 35, 32, 35, 35, 26, 36, 43, 45, 62, 165, 163, 179, 165, 158] 99


         1      -11622.7687             +nan
         2       -8649.8001       +2972.9686
         3       -7729.2510        +920.5490
         4       -7197.2997        +531.9514
         5       -6812.4444        +384.8552
         6       -6618.6592        +193.7852
         7       -6527.1121         +91.5472
         8       -6456.0109         +71.1012
         9       -6380.8885         +75.1224
        10       -6335.1357         +45.7528
        11       -6256.4901         +78.6455
        12       -6237.6818         +18.8083
        13       -6228.4983          +9.1835
        14       -6222.2958          +6.2025
        15       -6213.9192          +8.3766
        16       -6202.6507         +11.2686
        17       -6186.8005         +15.8501
        18       -6148.9835         +37.8170
        19       -6136.2139         +12.7696
        20       -6098.2966         +37.9173
        21       -6079.4377         +18.8588
        22       -6076.0796          +3.3581
        23

Training done


        45       -6048.2834          +0.0161
        46       -6048.2717          +0.0117
        47       -6048.2629          +0.0089


## Models cho 'đã' 2x3

In [121]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=2*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0 ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],])

Xct = np.concatenate(trainset['da'])
lengths = list([len(x) for x in trainset['da']])
print("training class", 'da')
print(Xct.shape, lengths, len(lengths))
hmm.fit(Xct, lengths=lengths)
models['da'] = hmm
print("Training done")

         1       -3815.7272             +nan
         2       -2928.1724        +887.5547

training class da
(1498, 1) [15, 14, 16, 14, 16, 26, 12, 13, 17, 13, 23, 13, 16, 14, 12, 19, 20, 24, 20, 20, 24, 20, 18, 26, 26, 22, 21, 21, 15, 20, 35, 25, 16, 18, 16, 26, 20, 22, 22, 30, 20, 21, 25, 24, 20, 21, 22, 22, 19, 21, 15, 20, 14, 16, 15, 21, 15, 16, 19, 12, 18, 18, 15, 24, 24, 22, 17, 15, 18, 19, 18, 15, 15, 18, 15, 11, 18, 27, 13] 79



         3       -2720.4096        +207.7629
         4       -2642.6648         +77.7448
         5       -2594.5370         +48.1278
         6       -2560.9596         +33.5775
         7       -2535.0658         +25.8938
         8       -2519.0658         +16.0000
         9       -2507.6540         +11.4118
        10       -2499.4185          +8.2355
        11       -2496.2786          +3.1399
        12       -2494.8642          +1.4144
        13       -2494.1606          +0.7036
        14       -2493.7716          +0.3890
        15       -2493.5270          +0.2446
        16       -2493.3532          +0.1738
        17       -2493.2173          +0.1358
        18       -2493.1035          +0.1138
        19       -2493.0029          +0.1006
        20       -2492.9104          +0.0925
        21       -2492.8224          +0.0880
        22       -2492.7365          +0.0859
        23       -2492.6507          +0.0858
        24       -2492.5637          +0.0870
        2

Training done



        52       -2490.4464          +0.0131
        53       -2490.4355          +0.0109
        54       -2490.4264          +0.0091


## Models cho 'được' 4x3

In [122]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=4*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xcth = np.concatenate(trainset['duoc'])
lengths = list([len(x) for x in trainset['duoc']])
print("training class", 'duoc')
print(Xcth.shape, lengths, len(lengths))
hmm.fit(Xcth, lengths=lengths)
models['duoc'] = hmm
print("Training done")

         1       -4537.0998             +nan
         2       -3145.5399       +1391.5599
         3       -2885.5056        +260.0343

training class duoc
(1796, 1) [22, 21, 27, 23, 30, 15, 18, 21, 30, 19, 20, 21, 30, 22, 14, 17, 23, 23, 19, 21, 18, 18, 20, 38, 21, 22, 35, 29, 18, 19, 22, 42, 22, 20, 24, 22, 21, 22, 22, 44, 29, 18, 14, 20, 24, 22, 19, 41, 24, 20, 18, 16, 20, 18, 21, 22, 30, 17, 21, 19, 16, 19, 18, 19, 16, 20, 14, 17, 16, 32, 26, 28, 24, 25, 25, 29, 21, 15, 17, 31] 80



         4       -2769.6502        +115.8553
         5       -2695.7283         +73.9219
         6       -2654.8869         +40.8415
         7       -2631.1867         +23.7001
         8       -2615.3488         +15.8379
         9       -2602.2201         +13.1287
        10       -2587.5223         +14.6978
        11       -2578.4185          +9.1038
        12       -2573.3537          +5.0647
        13       -2570.2144          +3.1394
        14       -2568.1666          +2.0478
        15       -2566.6393          +1.5273
        16       -2565.3619          +1.2773
        17       -2564.3144          +1.0476
        18       -2563.4970          +0.8174
        19       -2562.8747          +0.6223
        20       -2562.4036          +0.4712
        21       -2562.0484          +0.3551
        22       -2561.7848          +0.2636
        23       -2561.5938          +0.1911
        24       -2561.4580          +0.1358
        25       -2561.3618          +0.0962
        2

Training done


        62       -2559.5966          +0.0103
        63       -2559.5878          +0.0088


## Models cho 'người' 4x3

In [123]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=5*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ], ])

Xcth = np.concatenate(trainset['nguoi'])
lengths = list([len(x) for x in trainset['nguoi']])
print("training class", 'nguoi')
print(Xcth.shape, lengths, len(lengths))
hmm.fit(Xcth, lengths=lengths)
models['nguoi'] = hmm
print("Training done")

         1       -4735.5255             +nan
         2       -3213.1900       +1522.3354


training class nguoi
(1892, 1) [20, 18, 23, 25, 25, 38, 20, 25, 27, 22, 23, 23, 20, 26, 23, 19, 28, 24, 25, 19, 22, 28, 19, 24, 28, 20, 20, 32, 20, 18, 20, 31, 22, 19, 24, 14, 22, 15, 13, 22, 19, 25, 18, 27, 20, 20, 24, 17, 22, 19, 22, 26, 20, 23, 35, 28, 22, 30, 25, 27, 26, 23, 31, 30, 21, 19, 18, 20, 22, 15, 26, 19, 22, 51, 37, 42, 29, 29, 19, 18] 80


         3       -2818.3079        +394.8821
         4       -2667.6194        +150.6885
         5       -2602.8861         +64.7333
         6       -2561.1316         +41.7545
         7       -2521.1024         +40.0292
         8       -2482.1065         +38.9960
         9       -2457.4034         +24.7031
        10       -2438.1813         +19.2221
        11       -2424.5943         +13.5869
        12       -2416.2868          +8.3075
        13       -2411.2959          +4.9910
        14       -2407.6875          +3.6084
        15       -2405.2511          +2.4364
        16       -2403.9517          +1.2993
        17       -2403.2619          +0.6898
        18       -2402.8145          +0.4474
        19       -2402.5047          +0.3098
        20       -2402.2888          +0.2159
        21       -2402.1320          +0.1567
        22       -2402.0090          +0.1231
        23       -2401.9065          +0.1025
        24       -2401.8210          +0.0854
        25

Training done


        31       -2401.5952          +0.0094


## Models cho 'trong' 6x3

In [124]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=3*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xvn = np.concatenate(trainset['trong'])
lengths = list([len(x) for x in trainset['trong']])
print("training class", 'trong')
print(Xvn.shape, lengths, len(lengths))
hmm.fit(Xvn, lengths=lengths)
models['trong'] = hmm
print("Training done")

training class trong
(1887, 1) [30, 26, 23, 29, 28, 31, 22, 16, 21, 22, 19, 20, 33, 28, 25, 24, 25, 23, 26, 19, 21, 38, 29, 34, 18, 22, 19, 23, 17, 19, 20, 23, 24, 21, 33, 20, 20, 25, 21, 21, 25, 25, 24, 19, 28, 30, 20, 22, 21, 19, 16, 22, 26, 17, 25, 18, 31, 20, 21, 28, 20, 20, 28, 21, 24, 21, 23, 29, 22, 22, 23, 39, 21, 28, 20, 21, 22, 24, 21, 23] 80


         1       -4605.7759             +nan
         2       -2789.1173       +1816.6586
         3       -2460.9186        +328.1987
         4       -2306.0920        +154.8266
         5       -2247.1503         +58.9417
         6       -2219.1612         +27.9891
         7       -2209.9650          +9.1962
         8       -2206.5670          +3.3980
         9       -2204.8764          +1.6906
        10       -2203.8147          +1.0617
        11       -2203.0276          +0.7871
        12       -2202.4392          +0.5884
        13       -2202.0371          +0.4021
        14       -2201.7786          +0.2585
        15       -2201.6127          +0.1659
        16       -2201.5027          +0.1100
        17       -2201.4266          +0.0761
        18       -2201.3717          +0.0549
        19       -2201.3307          +0.0410
        20       -2201.2993          +0.0313
        21       -2201.2749          +0.0244
        22       -2201.2556          +0.0193
        23

Training done



        24       -2201.2274          +0.0127
        25       -2201.2168          +0.0106
        26       -2201.2077          +0.0091


# Test

In [125]:
print("Testing")
n_correct = {'cothe': 0, 'da': 0, 'duoc': 0, 'nguoi': 0,'trong': 0}
for true_cname in class_names:
    for O in testset[true_cname]:
        score = {cname: model.score(O, [len(O)]) for cname, model in models.items()}
        if (true_cname == max(score, key=score.get)): n_correct[true_cname] += 1
        print(true_cname, score, 'predict:', max(score, key=score.get))

Testing
cothe {'cothe': -458.02490440437685, 'da': -495.95844033566755, 'duoc': -1389.5021428063194, 'nguoi': -inf, 'trong': -inf} predict: cothe
cothe {'cothe': -129.06660181123752, 'da': -673.3563969359798, 'duoc': -460.16259664544805, 'nguoi': -inf, 'trong': -inf} predict: cothe
cothe {'cothe': -138.540146302178, 'da': -476.6962293011383, 'duoc': -384.336633264922, 'nguoi': -inf, 'trong': -inf} predict: cothe
cothe {'cothe': -237.29584303831246, 'da': -1772.6718921389713, 'duoc': -589.3723049986442, 'nguoi': -inf, 'trong': -inf} predict: cothe
cothe {'cothe': -280.19511406418496, 'da': -3147.7149840613147, 'duoc': -824.2221976484922, 'nguoi': -inf, 'trong': -inf} predict: cothe
cothe {'cothe': -334.5937797495441, 'da': -1838.670407985057, 'duoc': -907.1176367247942, 'nguoi': -inf, 'trong': -inf} predict: cothe
cothe {'cothe': -280.7441348853145, 'da': -1761.8385104412887, 'duoc': -1025.658184901991, 'nguoi': -inf, 'trong': -inf} predict: cothe
cothe {'cothe': -282.74073672470263, 'd

In [126]:
for cname in class_names:
    print('Accuracy:', cname, n_correct[cname]/n_test[cname])

Accuracy: cothe 1.0
Accuracy: da 0.85
Accuracy: duoc 0.65
Accuracy: nguoi 0.95
Accuracy: trong 0.7


In [127]:
print('All Accuracy:', sum(n_correct.values())/sum(n_test.values()))

All Accuracy: 0.8380952380952381


In [128]:
print(n_test)

{'cothe': 25, 'da': 20, 'duoc': 20, 'nguoi': 20, 'trong': 20}


In [129]:
#np.around(models['trong'].transmat_, 2)